In [33]:
#barcode 
#we also have to convert our category into vect
import numpy as np
import pandas as pd 
from sklearn.model_selection import GridSearchCV

In [2]:
mega=pd.read_csv("mega_chemical_harmness_category.csv")
mega.shape

(833, 2)

In [3]:
mega_cleaned=mega.drop_duplicates()
mega_cleaned.shape

(775, 2)

In [4]:
data= mega_cleaned.sample(frac=1, random_state=42).reset_index(drop=True)


In [5]:
data.shape

(775, 2)

In [6]:
data.head()

,ingredients,category
0,"Zeolite""",household
1,"BHA""",food
2,"Benzoyl Peroxide""",personal_care
3,"Glycine""",personal_care
4,"Fragrance""",cleaning


In [7]:
data.shape

(775, 2)

In [8]:
data['category'].unique()

array(['household', 'food', 'personal_care', 'cleaning', 'cosmetic',
       'stationery', 'medicine'], dtype=object)

In [9]:
df=data.drop_duplicates()
df.shape

(775, 2)

In [10]:
df['category'] = df['category'].str.replace('"', '')

In [11]:
df['category'].value_counts()

category
food             160
medicine         141
cosmetic         116
personal_care     99
cleaning          94
stationery        84
household         81
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [13]:
X = df["ingredients"]
y = df["category"]

In [14]:
tfid=TfidfVectorizer( ngram_range=(1, 3), stop_words='english', max_features=15000, min_df=2, max_df=0.9 )
X_vec= tfid.fit_transform(X)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y) 

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, stratify=y,random_state=42)

In [16]:
#we will find top 3 classificatio algo for ensemble technique. we will use proba and mojority of it 

In [17]:
#loggistic regression
lr = LogisticRegression(max_iter=1000, class_weight="balanced")
lr.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [18]:

# proba = lr.predict_proba(X_test)[0]
# max_confidence = np.max(proba)
# predicted_index = np.argmax(proba)
# predicted_label = le.inverse_transform([predicted_index])[0]
# if max_confidence< 50:
#     print("others")
# else:
#     predicted_label, max_confidence

In [19]:
y_pred=lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.68      0.70        19
           1       0.36      0.17      0.24        23
           2       0.55      0.38      0.44        32
           3       0.33      0.25      0.29        16
           4       0.37      0.89      0.53        28
           5       0.44      0.35      0.39        20
           6       0.78      0.41      0.54        17

    accuracy                           0.46       155
   macro avg       0.51      0.45      0.45       155
weighted avg       0.50      0.46      0.45       155



In [20]:
#Random_Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators=400,
    min_samples_split=4,
    min_samples_leaf=1,
    max_features='log2',
    max_depth=None,
    bootstrap=False,
    random_state=42
)

In [21]:
rf.fit(X_train, y_train)

,n_estimators,400
,criterion,'gini'
,max_depth,None
,min_samples_split,4
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'log2'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,False
,oob_score,False


In [22]:
y_pred=rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.37      0.50        19
           1       0.30      0.13      0.18        23
           2       0.42      0.41      0.41        32
           3       0.40      0.12      0.19        16
           4       0.34      0.89      0.49        28
           5       0.40      0.30      0.34        20
           6       0.82      0.53      0.64        17

    accuracy                           0.42       155
   macro avg       0.49      0.39      0.39       155
weighted avg       0.47      0.42      0.40       155



In [23]:
#svc
from sklearn.svm import SVC
svc=SVC(random_state=42)
svc.fit(X_train, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [24]:
y_pred=svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.32      0.46        19
           1       0.33      0.17      0.23        23
           2       0.35      0.56      0.43        32
           3       0.40      0.12      0.19        16
           4       0.45      0.89      0.60        28
           5       0.47      0.35      0.40        20
           6       0.78      0.41      0.54        17

    accuracy                           0.45       155
   macro avg       0.52      0.40      0.41       155
weighted avg       0.49      0.45      0.42       155



In [25]:
#Xgboost
import xgboost as xgb
xg=xgb.XGBClassifier(objective="multi:softmax", random_state=42)
xg.fit(X_train, y_train)

,objective,'multi:softmax'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [26]:
y_pred=xg.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.21      0.33        19
           1       0.25      0.04      0.07        23
           2       0.46      0.41      0.43        32
           3       0.20      0.06      0.10        16
           4       0.27      0.96      0.42        28
           5       0.56      0.25      0.34        20
           6       0.75      0.18      0.29        17

    accuracy                           0.35       155
   macro avg       0.47      0.30      0.28       155
weighted avg       0.45      0.35      0.30       155



In [27]:
# from lightgbm import LGBMClassifier
# lg=LGBMClassifier(objective='multiclass', metric= 'multi_logloss', learning_rate= 0.05, random_state=42 )
# lg.fit(X_train,y_train)

In [34]:
y_pred=lg.predict(X_test)
print(classification_report(y_test, y_pred))

NameError: name 'lg' is not defined

In [35]:
#catboost
from catboost import CatBoostClassifier

cat = CatBoostClassifier( iterations=100,learning_rate=0.03,         
    depth=6,                    # Depth of the trees
    loss_function='MultiClass',    # Loss function for binary classification (or 'MultiClass' for multi-class)
    eval_metric='Accuracy', random_seed=42             
)

cat.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.2322581	test: 0.2258065	best: 0.2258065 (0)	total: 156ms	remaining: 15.4s
1:	learn: 0.2370968	test: 0.2258065	best: 0.2258065 (0)	total: 172ms	remaining: 8.44s
2:	learn: 0.2306452	test: 0.2193548	best: 0.2258065 (0)	total: 181ms	remaining: 5.86s
3:	learn: 0.2387097	test: 0.2193548	best: 0.2258065 (0)	total: 191ms	remaining: 4.58s
4:	learn: 0.2419355	test: 0.2193548	best: 0.2258065 (0)	total: 200ms	remaining: 3.8s
5:	learn: 0.2435484	test: 0.2193548	best: 0.2258065 (0)	total: 209ms	remaining: 3.27s
6:	learn: 0.2306452	test: 0.2129032	best: 0.2258065 (0)	total: 216ms	remaining: 2.87s
7:	learn: 0.2548387	test: 0.2387097	best: 0.2387097 (7)	total: 224ms	remaining: 2.57s
8:	learn: 0.2290323	test: 0.2129032	best: 0.2387097 (7)	total: 231ms	remaining: 2.33s
9:	learn: 0.2274194	test: 0.2129032	best: 0.2387097 (7)	total: 236ms	remaining: 2.13s
10:	learn: 0.2258065	test: 0.2129032	best: 0.2387097 (7)	total: 244ms	remaining: 1.98s
11:	learn: 0.2209677	test: 0.2129032	best: 0.2387097 (

In [ ]:
y_pred=cat.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
#hyper parameter tuning 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# 1. logist regression
lr_param_grid = {
    "penalty": ["l2"],              
    "C": [0.01, 0.1, 1, 10, 100], 
    "solver": ["lbfgs", "saga"],   
    "class_weight": [None, "balanced"],
    "max_iter": [200, 500, 700]
}

lr_grid = GridSearchCV(
    estimator=LogisticRegression(random_state=42, n_jobs=-1),
    param_grid=lr_param_grid,
    scoring="f1_weighted",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    refit=True
)
lr_grid.fit(X_train, y_train)


In [ ]:
print("Logistic Regression - Best params:", lr_grid.best_params_)
print("Best CV score:", lr_grid.best_score_)

# Predict
y_pred_lr = lr_grid.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, y_pred_lr))
print("Classification report:\n", classification_report(y_test, y_pred_lr))

In [29]:
# 2. SVC
svc_param_grid = {
    "C": [0.1, 1, 10],
    "kernel": ["rbf", "poly"],
    "degree": [2, 3],  
    "gamma": ["scale", "auto"],
    "class_weight": [None, "balanced"]
}

svc_grid = GridSearchCV(
    estimator=SVC(probability=True, random_state=42),
    param_grid=svc_param_grid,
    scoring="f1_weighted",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    refit=True
)

svc_grid.fit(X_train, y_train)

NameError: name 'GridSearchCV' is not defined

In [30]:
print("SVC - Best params:", svc_grid.best_params_)
print("Best CV score:", svc_grid.best_score_)

# Predict
y_pred_svc = svc_grid.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, y_pred_svc))
print("Classification report:\n", classification_report(y_test, y_pred_svc))


NameError: name 'svc_grid' is not defined

In [31]:
# random forest
rf_param_grid = {
    "n_estimators": [100, 300, 500],
    "max_depth": [None, 10, 30],
    "min_samples_split": [2, 5, 6],
    "min_samples_leaf": [1, 2, 4],
    "class_weight": [None, "balanced"],
    "bootstrap": [True, False]
}

rf_grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    scoring="f1_weighted",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    refit=True
)
rf_grid.fit(X_train,y_train)

NameError: name 'GridSearchCV' is not defined

In [32]:
print("Random Forest - Best params:", rf_grid.best_params_)
print("Best CV score:", rf_grid.best_score_)

y_pred_rf = rf_grid.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification report:\n", classification_report(y_test, y_pred_rf))

NameError: name 'rf_grid' is not defined

In [36]:
#retrain on best parameters
lr = LogisticRegression(C= 1, class_weight='balanced', max_iter= 200, penalty='l2', solver= 'lbfgs')
lr.fit(X_train, y_train)

rf = RandomForestClassifier( bootstrap= True,class_weight= None,max_depth= None,min_samples_leaf= 1, min_samples_split= 5,n_estimators= 300,
                            max_features='log2',random_state=42)
rf.fit(X_train, y_train)

svc=SVC(C= 1,class_weight= 'balanced', degree= 2, gamma= 'scale', kernel= 'rbf')
svc.fit(X_train,y_train)

,C,1
,kernel,'rbf'
,degree,2
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [37]:
#preiction
y_pred_lr=lr.predict(X_test)
y_pred_rf=rf.predict(X_test)
y_pred_svc=svc.predict(X_test)

In [38]:
#ensemble voting
from collections import Counter
import numpy as np

def majority_vote(preds_list, tie_breaker_order=None):
    preds_array = np.vstack(preds_list).T 
    n_samples = preds_array.shape[0]
    ensemble = []

    for i in range(n_samples):
        votes = preds_array[i]
        cnt = Counter(votes)
        most_common = cnt.most_common()
        # Check for strict majority (> half of models, here with 3 models means count >=2)
        top_vote, top_count = most_common[0]
        if top_count >= 2:
            ensemble.append(top_vote)
        else:
            if tie_breaker_order is not None:
                
                for idx in tie_breaker_order:
                    ensemble.append(votes[idx])
                    break
            else:
                # fallback: take first model's prediction
                ensemble.append(votes[0])

    return np.array(ensemble)


# Example usage:
# y_pred_lr = lr.predict(X_test)
# y_pred_rf = rf.predict(X_test)
# y_pred_svc = svc.predict(X_test)

# Define tie-breaker preference: prefer RandomForest over LogisticRegression over SVC
# indices correspond to order in the list below: [rf, lr, svc]
ensemble_preds = majority_vote(
    [y_pred_rf, y_pred_lr, y_pred_svc],
    tie_breaker_order=[0, 1, 2]
)

# Evaluate
from sklearn.metrics import classification_report, accuracy_score
print("Ensemble accuracy:", accuracy_score(y_test, ensemble_preds))
print("Ensemble report:\n", classification_report(y_test, ensemble_preds))


Ensemble accuracy: 0.47096774193548385
Ensemble report:
               precision    recall  f1-score   support

           0       0.75      0.63      0.69        19
           1       0.38      0.22      0.28        23
           2       0.54      0.41      0.46        32
           3       0.40      0.25      0.31        16
           4       0.37      0.89      0.52        28
           5       0.47      0.35      0.40        20
           6       0.78      0.41      0.54        17

    accuracy                           0.47       155
   macro avg       0.53      0.45      0.46       155
weighted avg       0.51      0.47      0.46       155



In [40]:
import joblib
import numpy as np
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# --- Ensemble class ---
class MajorityVotingEnsemble:
    def __init__(self, models, tie_breaker_order=None):
        self.models = models
        self.tie_breaker_order = (
            tie_breaker_order if tie_breaker_order is not None else list(range(len(models)))
        )

    def predict(self, X):
        preds = [m.predict(X) for m in self.models]
        preds_array = np.vstack(preds).T  # (n_samples, n_models)
        ensemble = []
        for votes in preds_array:
            cnt = Counter(votes)
            top_vote, top_count = cnt.most_common(1)[0]
            if top_count >= 2:
                ensemble.append(top_vote)
            else:
                # no majority: fallback to tie_breaker_order preference
                for idx in self.tie_breaker_order:
                    ensemble.append(votes[idx])
                    break
        return np.array(ensemble)

# --- Replace these with your already trained/tuned models ---
# Example placeholders; in your code use the fitted lr, rf, svc
# lr = LogisticRegression(**best_lr_params).fit(X_train, y_train)
# rf = RandomForestClassifier(**best_rf_params).fit(X_train, y_train)
# svc = SVC(**best_svc_params, probability=True).fit(X_train, y_train)

# For demonstration assume lr, rf, svc exist and are fitted:
# ensemble creation
ensemble = MajorityVotingEnsemble(models=[rf, lr, svc], tie_breaker_order=[0,1,2])

# --- Save ---
joblib.dump(rf, "random_forest.pkl")
joblib.dump(lr, "logistic_regression.pkl")
joblib.dump(svc, "svc.pkl")
joblib.dump(ensemble, "majority_ensemble.pkl")

loaded_ensemble = joblib.load("majority_ensemble.pkl")
y_pred = loaded_ensemble.predict(X_test)  # your test or new data

print("Ensemble accuracy:", accuracy_score(y_test, y_pred))


Ensemble accuracy: 0.47096774193548385


In [41]:
joblib.dump(le,"label_encoder_.pkl")

['label_encoder_.pkl']

In [42]:
from joblib import dump
dump(ensemble,'majority_ensemble.joblib')
dump(le, "label_encode_main.joblib")

['label_encode_main.joblib']

In [43]:
dump(tfid,"tfidf_vectorizer_main.joblib")

['tfidf_vectorizer_main.joblib']

In [44]:
import joblib
from ensemble import MajorityVotingEnsemble  # imports the class
# assume lr, rf, svc are already trained/fitted and y_test etc exist

# create ensemble (you can give weights or leave defaults)
ensemble = MajorityVotingEnsemble(models=[rf, lr, svc], tie_breaker_order=[0,1,2])

# optional: evaluate immediately
y_pred = ensemble.predict(X_test)
print("Ensemble accuracy:", accuracy_score(y_test, y_pred))

# save base models and the ensemble
joblib.dump(rf, "rf.pkl")
joblib.dump(lr, "lr.pkl")
joblib.dump(svc, "svc.pkl")
joblib.dump(ensemble, "majority_ensemble.pkl")


Ensemble accuracy: 0.4838709677419355


['majority_ensemble.pkl']